In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import time 
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from tensorflow.keras.models import Sequential, load_model
from keras.layers import LSTM, Dense
from tensorflow import keras
import tensorflow as tf
import csv
from sklearn.decomposition import PCA
import sklearn
import pickle
from sklearn.model_selection import train_test_split
%load_ext tensorboard

In [2]:
import glob
files = glob.glob("../../datasets/camel-1.6.csv", recursive = True)
ant_master = pd.concat(map(pd.read_csv, files))

In [3]:
ant_master

,name,version,name.1,wmc,dit,noc,cbo,rfc,lcom,ca,...,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc,bug
0,camel,1.6,org.apache.camel.component.cxf.CxfComponent,5,3,0,7,10,0,1,...,1.0,1,0.921053,0.360000,1,2,7.400000,1,0.6000,0
1,camel,1.6,org.apache.camel.impl.DefaultExchangeFormatter,4,1,0,3,5,4,1,...,1.0,1,0.000000,0.500000,0,0,3.000000,1,0.5000,0
2,camel,1.6,org.apache.camel.component.jetty.JettyHttpComp...,20,4,0,26,95,144,2,...,1.0,0,0.727273,0.197368,4,5,20.300000,3,1.0000,0
3,camel,1.6,org.apache.camel.language.simple.SimpleLanguage,3,2,0,8,22,3,2,...,0.0,0,0.750000,0.666667,1,3,54.000000,15,5.3333,1
4,camel,1.6,org.apache.camel.util.ServiceHelper,8,1,0,25,20,22,22,...,1.0,0,0.000000,0.250000,0,0,20.875000,1,0.7500,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
960,camel,1.6,org.apache.camel.util.concurrent.CountingLatch,6,1,0,4,13,0,3,...,1.0,1,0.000000,0.444444,0,0,6.500000,1,0.8333,1
961,camel,1.6,org.apache.camel.component.mail.MailUtils,3,1,0,4,19,3,4,...,0.0,0,0.000000,0.333333,0,0,61.333333,9,5.6667,0
962,camel,1.6,org.apache.camel.component.uface.swing.SwingBr...,3,2,0,8,16,3,1,...,0.0,0,0.875000,0.750000,0,0,13.666667,1,0.3333,0
963,camel,1.6,org.apache.camel.component.jpa.QueryBuilder,12,1,3,8,25,46,6,...,0.0,1,0.000000,0.226190,0,0,7.416667,2,1.1667,0


In [4]:
train, test = train_test_split(ant_master, test_size=0.2)

In [5]:
cols_to_norm = ['wmc', 'dit', 'noc', 'cbo', 'rfc', 'lcom', 'ca', 'ce', 'npm', 'lcom3', 'loc', 'dam', 'moa', 'mfa', 'cam', 'ic', 'cbm', 'amc', 'max_cc', 'avg_cc']
x_train = train[cols_to_norm]
x_test = test[cols_to_norm]

In [6]:
x_train

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
740,5,1,0,1,6,6,1,0,5,0.750000,24,1.0,0,0.000000,0.700000,0,0,3.400000,1,0.8000
272,3,1,0,8,22,3,5,3,2,2.000000,193,0.0,0,0.000000,0.583333,0,0,63.333333,10,5.6667
305,19,3,0,16,26,157,4,15,18,0.944444,111,1.0,0,0.651163,0.215278,1,1,4.526316,1,0.7895
208,5,2,0,9,10,10,3,6,5,2.000000,30,0.0,0,0.926829,0.360000,1,1,5.000000,1,0.6000
840,5,1,0,4,5,10,4,0,5,2.000000,5,0.0,0,0.000000,1.000000,0,0,0.000000,1,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105,11,3,0,4,15,35,1,4,10,0.200000,62,1.0,0,0.795455,0.590909,2,5,4.545455,1,0.8182
435,4,3,0,5,6,0,1,4,3,0.333333,21,1.0,0,0.750000,0.500000,1,1,4.000000,1,0.7500
188,2,3,0,6,11,1,0,6,1,2.000000,39,0.0,0,0.972222,0.666667,1,2,18.500000,1,0.5000
569,22,3,1,15,66,193,4,14,16,0.849206,318,1.0,1,0.716216,0.295455,1,3,13.181818,4,1.3636


In [7]:
x_test

,wmc,dit,noc,cbo,rfc,lcom,ca,ce,npm,lcom3,loc,dam,moa,mfa,cam,ic,cbm,amc,max_cc,avg_cc
326,3,3,0,10,15,1,2,9,3,2.000000,54,0.0,0,0.913043,0.500000,0,0,17.000000,2,1.0000
404,3,5,0,5,4,0,2,3,3,0.500000,21,1.0,2,0.894737,0.500000,0,0,5.333333,1,0.6667
627,6,0,0,8,23,0,1,7,1,0.700000,100,1.0,0,0.000000,0.366667,0,0,15.000000,4,1.1667
258,6,1,0,14,28,7,2,12,3,0.666667,213,1.0,0,0.500000,0.400000,1,1,34.000000,1,0.6667
538,36,2,0,21,49,590,5,19,36,0.963810,261,1.0,2,0.406780,0.160494,1,2,5.833333,2,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
805,4,3,0,1,5,4,0,1,3,0.666667,16,1.0,1,0.964286,0.666667,1,1,2.750000,1,0.5000
821,3,1,0,5,11,0,1,4,3,0.000000,53,1.0,0,0.000000,0.500000,0,0,16.333333,1,0.6667
350,32,3,0,38,97,320,3,37,20,0.854839,834,1.0,1,0.546875,0.175953,2,3,24.687500,7,1.7813
182,2,3,0,3,5,1,1,3,1,2.000000,17,0.0,0,0.972222,0.666667,1,2,7.500000,1,0.5000


In [8]:
y_train = train['bug']
y_test = test['bug']

In [9]:
y_train

740    0
272    0
305    0
208    0
840    0
      ..
105    0
435    0
188    0
569    0
722    1
Name: bug, Length: 772, dtype: int64

In [10]:
y_test

326    1
404    0
627    0
258    0
538    0
      ..
805    0
821    0
350    0
182    0
37     0
Name: bug, Length: 193, dtype: int64

In [11]:
x_train_np = np.array(x_train)
x_test_np = np.array(x_test)

In [12]:
x_train_np

array([[ 5.        ,  1.        ,  0.        , ...,  3.4       ,
         1.        ,  0.8       ],
       [ 3.        ,  1.        ,  0.        , ..., 63.33333333,
        10.        ,  5.6667    ],
       [19.        ,  3.        ,  0.        , ...,  4.52631579,
         1.        ,  0.7895    ],
       ...,
       [ 2.        ,  3.        ,  0.        , ..., 18.5       ,
         1.        ,  0.5       ],
       [22.        ,  3.        ,  1.        , ..., 13.18181818,
         4.        ,  1.3636    ],
       [ 5.        ,  1.        ,  5.        , ...,  8.4       ,
         2.        ,  1.        ]])

In [13]:
x_test_np

array([[ 3.        ,  3.        ,  0.        , ..., 17.        ,
         2.        ,  1.        ],
       [ 3.        ,  5.        ,  0.        , ...,  5.33333333,
         1.        ,  0.6667    ],
       [ 6.        ,  0.        ,  0.        , ..., 15.        ,
         4.        ,  1.1667    ],
       ...,
       [32.        ,  3.        ,  0.        , ..., 24.6875    ,
         7.        ,  1.7813    ],
       [ 2.        ,  3.        ,  0.        , ...,  7.5       ,
         1.        ,  0.5       ],
       [ 1.        ,  1.        ,  0.        , ...,  0.        ,
         1.        ,  1.        ]])

In [14]:
train_x = np.reshape(x_train_np, (x_train_np.shape[0], 1, x_train_np.shape[1]))
test_x = np.reshape(x_test_np, (x_test_np.shape[0], 1, x_test_np.shape[1]))

In [15]:
train_y = y_train.to_numpy()
test_y = y_test.to_numpy()

In [16]:
train_x

array([[[ 5.        ,  1.        ,  0.        , ...,  3.4       ,
          1.        ,  0.8       ]],

       [[ 3.        ,  1.        ,  0.        , ..., 63.33333333,
         10.        ,  5.6667    ]],

       [[19.        ,  3.        ,  0.        , ...,  4.52631579,
          1.        ,  0.7895    ]],

       ...,

       [[ 2.        ,  3.        ,  0.        , ..., 18.5       ,
          1.        ,  0.5       ]],

       [[22.        ,  3.        ,  1.        , ..., 13.18181818,
          4.        ,  1.3636    ]],

       [[ 5.        ,  1.        ,  5.        , ...,  8.4       ,
          2.        ,  1.        ]]])

In [17]:
test_y

array([ 1,  0,  0,  0,  0,  0,  0,  0,  4,  0,  1,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  1,  0,  1,  0,  1,  0,  0,  2,  2,  0,  0, 11,  0,
        0,  0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  2,  3,  0,  7,  2,  1,  0,  0,  0,  0,  0,
        0,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  3,  3,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  4, 28,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  1,  0,  0,
        0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  1,  0,  0,  1,  1,  0,  0,
        0,  1,  0,  0,  0,  0,  0,  0,  1,  1,  0,  1,  0,  0,  0,  0,  0,
        2,  0,  0,  1,  1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  2,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0])

In [18]:
# Designing and initializing the model.
from keras.layers import LSTM, Dense, SimpleRNN
# Designing and initializing the model.
model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(50, dropout = 0.2, return_sequences=True))
model.add(LSTM(20, return_sequences=False))
model.add(Dense(1, activation = 'relu'))
model.compile(loss = 'mse' , optimizer = keras.optimizers.Adam(learning_rate=0.0005), metrics = ['mse', 'mae', tf.keras.metrics.RootMeanSquaredError(), tf.keras.metrics.MeanSquaredLogarithmicError()] )

In [19]:
# Fitting the model on training data.
import datetime
log_dir = '/home/bavanya/Desktop/tensorboard/' + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
history = model.fit(train_x, train_y, epochs = 100, callbacks=[tensorboard_callback])

Epoch 1/100
25/25 [==============================] - 5s 16ms/step - loss: 2.9640 - mse: 2.9640 - mae: 0.5629 - root_mean_squared_error: 1.7216 - mean_squared_logarithmic_error: 0.2799
Epoch 2/100
25/25 [==============================] - 0s 5ms/step - loss: 2.7730 - mse: 2.7730 - mae: 0.7073 - root_mean_squared_error: 1.6652 - mean_squared_logarithmic_error: 0.2556
Epoch 3/100
25/25 [==============================] - 0s 6ms/step - loss: 2.6701 - mse: 2.6701 - mae: 0.7861 - root_mean_squared_error: 1.6340 - mean_squared_logarithmic_error: 0.2725
Epoch 4/100
25/25 [==============================] - 0s 6ms/step - loss: 2.5994 - mse: 2.5994 - mae: 0.7291 - root_mean_squared_error: 1.6123 - mean_squared_logarithmic_error: 0.2537
Epoch 5/100
25/25 [==============================] - 0s 6ms/step - loss: 2.5132 - mse: 2.5132 - mae: 0.7120 - root_mean_squared_error: 1.5853 - mean_squared_logarithmic_error: 0.2471
Epoch 6/100
25/25 [==============================] - 0s 6ms/step - loss: 2.3886 - ms

25/25 [==============================] - 0s 5ms/step - loss: 1.9011 - mse: 1.9011 - mae: 0.5147 - root_mean_squared_error: 1.3788 - mean_squared_logarithmic_error: 0.1603
Epoch 89/100
25/25 [==============================] - 0s 5ms/step - loss: 1.5402 - mse: 1.5402 - mae: 0.5687 - root_mean_squared_error: 1.2410 - mean_squared_logarithmic_error: 0.1703
Epoch 90/100
25/25 [==============================] - 0s 6ms/step - loss: 1.4778 - mse: 1.4778 - mae: 0.5172 - root_mean_squared_error: 1.2157 - mean_squared_logarithmic_error: 0.1495
Epoch 91/100
25/25 [==============================] - 0s 5ms/step - loss: 1.4161 - mse: 1.4161 - mae: 0.5209 - root_mean_squared_error: 1.1900 - mean_squared_logarithmic_error: 0.1483
Epoch 92/100
25/25 [==============================] - 0s 6ms/step - loss: 1.4038 - mse: 1.4038 - mae: 0.5141 - root_mean_squared_error: 1.1848 - mean_squared_logarithmic_error: 0.1515
Epoch 93/100
25/25 [==============================] - 0s 6ms/step - loss: 1.8696 - mse: 1.869

In [20]:
%tensorboard --logdir '/home/bavanya/Desktop/tensorboard/'

Reusing TensorBoard on port 6006 (pid 3750), started 3:33:36 ago. (Use '!kill 3750' to kill it.)

In [21]:
score = model.evaluate(test_x, test_y)
score

7/7 [==============================] - 1s 2ms/step - loss: 5.3727 - mse: 5.3727 - mae: 0.6757 - root_mean_squared_error: 2.3179 - mean_squared_logarithmic_error: 0.2488


[5.372740268707275,
 5.372740268707275,
 0.6757391095161438,
 2.3179171085357666,
 0.2488439530134201]